In [1]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the CSV file
file_path = '/kaggle/input/petfinder-adoption-prediction/train/train.csv'
data = pd.read_csv(file_path)

data

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,NaN,2,266,0,3,1,0,0,2,...,1,4,0,41326,61c84bd7bcb6fb31d2d480b1bcf9682e,0,I have 4 kittens that need to be adopt urgentl...,dc0935a84,3.0,2
14989,2,Serato & Eddie,60,265,264,3,1,4,7,2,...,1,2,0,41326,1d5096c4a5e159a3b750c5cfcf6ceabf,0,Serato(female cat- 3 color) is 4 years old and...,a01ab5b30,3.0,4
14990,2,Monkies,2,265,266,3,5,6,7,3,...,1,5,30,41326,6f40a7acfad5cc0bb3e44591ea446c05,0,"Mix breed, good temperament kittens. Love huma...",d981b6395,5.0,3
14991,2,Ms Daym,9,266,0,2,4,7,0,1,...,1,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4


In [2]:
# Store the 'PetID' column in a separate DataFrame
pet_id_df = data[['PetID']]

# Drop non-numeric columns from the dataset
data = data.select_dtypes(include=['number'])

# Define the features (X) and target variable (y)
X = data.drop(columns=['AdoptionSpeed'])  # Replace 'target_column' with your target column name
y = data['AdoptionSpeed']

# Split the data into training and validation sets (70% train, 30% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Create the LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# Define the parameters for the LightGBM model
params = {
    'objective': 'regression',  # Change to 'binary' for binary classification, or 'multiclass' for multiclass classification
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'verbose': 1
}

# Train the LightGBM model
model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[train_data, val_data]
    # early_stopping_rounds=50,
    # verbose_eval=100
)

# Predict on the validation set
y_pred = model.predict(X_val, num_iteration=model.best_iteration)

y_pred

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003326 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 425
[LightGBM] [Info] Number of data points in the train set: 10495, number of used features: 19
[LightGBM] [Info] Start training from score 2.517675


array([3.65864704, 2.2487915 , 3.3566938 , ..., 2.95199315, 2.73421406,
       3.13595243])

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate evaluation metrics
rmse = mean_squared_error(y_val, y_pred, squared=False)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

# Print the evaluation metrics
print(f'Validation RMSE: {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R2): {r2}')

Validation RMSE: 1.1070778900809994
Mean Absolute Error (MAE): 0.9100774572710502
R-squared (R2): 0.1334283410048306


In [4]:
# Load the Test file
file_path = '/kaggle/input/petfinder-adoption-prediction/test/test.csv'
test_data = pd.read_csv(file_path)

test_data

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt
0,2,Dopey & Grey,8,266,266,1,2,6,7,1,...,2,1,2,0,41326,2ece3b2573dcdcebd774e635dca15fd9,0,"Dopey Age: 8mths old Male One half of a pair, ...",e2dfc2935,2.0
1,2,Chi Chi,36,285,264,2,1,4,7,2,...,1,2,1,0,41326,2ece3b2573dcdcebd774e635dca15fd9,0,"Please note that Chichi has been neutered, the...",f153b465f,1.0
2,2,Sticky,2,265,0,1,6,7,0,2,...,2,1,1,200,41326,e59c106e9912fa30c898976278c2e834,0,"Sticky, named such because of his tendency to ...",3c90f3f54,4.0
3,1,Dannie & Kass [In Penang],12,307,0,2,2,5,0,2,...,1,1,2,0,41326,e59c106e9912fa30c898976278c2e834,0,Dannie and Kass are mother and daughter. We en...,e02abc8a3,5.0
4,2,Cuddles,12,265,0,1,2,3,7,2,...,1,1,1,0,41326,e59c106e9912fa30c898976278c2e834,0,"Extremely cuddly cat, hence the origin of his ...",09f0df7d1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967,1,Hugo,5,307,307,1,1,2,0,2,...,2,1,2,150,41401,18ec8ca4486bc2760de0bd5390cee30c,0,Found on the streets. Treated for mange. They ...,ae57f8d52,5.0
3968,1,Spot,30,307,307,1,1,2,7,2,...,2,1,1,0,41326,d83be5f5e2d04e24d7376e99eafd8708,0,Very good guard dog. Healthy was found in Fron...,83432904d,2.0
3969,2,NaN,5,300,0,3,1,2,4,2,...,2,1,6,0,41401,30aa45fdbe45e39d5614ef583b569073,0,these cat's mother was killed when they was ne...,399013029,1.0
3970,1,Smokey,24,307,0,2,5,7,0,2,...,2,1,1,0,41325,087903c2819a6297519c93d962b488d5,0,"smokey is good family pet. very obedient,so lo...",fd80b8c80,3.0


In [5]:
# Store the 'PetID' column in a separate DataFrame
test_pet_id_df = test_data[['PetID']]

# Drop non-numeric columns from the test dataset
test_data_numeric = test_data.select_dtypes(include=['number'])

# If needed, fill or drop missing values (example: fill with 0)
# test_data_numeric.fillna(0, inplace=True)

# Predict on the test set
y_test_pred = model.predict(test_data_numeric, num_iteration=model.best_iteration)

y_test_pred

array([2.09101289, 2.01300106, 2.07141014, ..., 3.58216004, 2.98695079,
       2.66912708])

In [6]:
import numpy as np

# Round the predictions to the nearest integer and clip values between 0 and 4
y_pred_rounded = np.clip(np.round(y_test_pred), 0, 4).astype(int)

# Convert y_pred_rounded to a DataFrame
predicted_df = pd.DataFrame(y_pred_rounded, columns=['AdoptionSpeed'])

# Concatenate pet_id_df with the predicted values DataFrame
predictions_df = pd.concat([test_pet_id_df.reset_index(drop=True), predicted_df], axis=1)

predictions_df

,PetID,AdoptionSpeed
0,e2dfc2935,2
1,f153b465f,2
2,3c90f3f54,2
3,e02abc8a3,3
4,09f0df7d1,3
...,...,...
3967,ae57f8d52,4
3968,83432904d,4
3969,399013029,4
3970,fd80b8c80,3


In [7]:
from datetime import datetime

# Rename columns to match the desired header
predictions_df.columns = ['PetID', 'AdoptionSpeed']

# Define the filename with the timestamp
filename =f'submission.csv'

# Save predictions_df to a CSV file
predictions_df.to_csv(filename, index=False)

print(f'Predictions saved to {filename}')


Predictions saved to submission.csv
